In [3]:
import import_ipynb
import math
from tqdm import tqdm

from postprocessing import *

In [4]:
def evaluate_binaries(labels, preds):
    
    assert(labels.size() == preds.size())
    
    size = labels.size()
    total = math.prod([i for i in size])
    
    labels, preds = labels.view((total)), preds.view((total))
    
    tp = 0
    fp = 0 
    tn = 0
    fn = 0
    events = 0
    noEvents = 0
    
    for i in range(total):
        
        x, y = labels[i], preds[i]
        
        if x == 1:
            
            events += 1
            
            if y == 1:
                tp += 1
            
            else:
                fn += 1
        
        else:
            
            noEvents += 1
            
            if y == 1:
                fp += 1
            
            else:
                tn += 1
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    print("{} evaluations in total.".format(total))
    print("{}/{} cells with earthquake activity were correctly identified. ({:.2f} %)".format(tp, events, tp * 100.0 / events))
    print("{}/{} cells with no earthquake activity were correctly identified. ({:.2f} %)".format(tn, noEvents, tn * 100.0 / noEvents))
    print("{}/{} positives turned out to be false.".format(fp, fp+tp))
    print("{}/{} negatives turned out to be false.".format(fn, fn+tn)) 
    print("Preicision: {:.3f}".format(precision))
    print("Recall: {:.3f}".format(recall))
    print("F1 Score: {:.3f}".format(f1))
    print("Overall accuracy: {:.2f} %".format((tp + tn) * 100.0 / total))

In [5]:
def evaluate(data, model, train_size, lookback_window, threshold=None, mode='binary'):
    
    a, b, c = data.size()
    
    train_size = int(train_size * a) - lookback_window
    test_size = a - train_size - lookback_window
    
    labels_train = torch.zeros((train_size, 1, c))
    preds_train = torch.zeros((train_size, 1, c))
    
    labels_test = torch.zeros((test_size, 1, c))
    preds_test = torch.zeros((test_size, 1, c))
    
    for i in tqdm(range(a - lookback_window)):
        
        X = data[i : i+lookback_window]
        y = data[i+lookback_window]
        
        out = model(X)
        
        if threshold:
            out = threshold_based_binaries(out, threshold)
        
        if i < train_size:
            labels_train[i] = y
            preds_train[i] = out
        
        else:
            labels_test[i-train_size] = y
            preds_test[i-train_size] = out
       
    print("Evaluation results for training dataset:")
    
    if mode == 'binary':
        evaluate_binaries(labels_train, preds_train)
    
    print("Evaluation results for training dataset:")
    
    if mode == 'binary':
        evaluate_binaries(labels_test, preds_test)
    